In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy, make_feature_set
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-08-04 15:55:36,585 logger 19 [INFO]    [logger_func] start 
2018-08-04 15:55:36,585 logger 19 [INFO]    [logger_func] start 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
app = pd.read_csv('../data/application_train_test.csv')

' NULL埋め '
app['DAYS_EMPLOYED'].fillna(-240, inplace=True)
app['OCCUPATION_TYPE'].fillna('XNA', inplace=True)

flag = {'N':0, 'Y':1}
app['FLAG_OWN_CAR'] = app['FLAG_OWN_CAR'].map(flag)
app['FLAG_OWN_REALTY'] = app['FLAG_OWN_REALTY'].map(flag)

In [20]:
cat = 'FLAG_OWN_CAR'
cat = 'FLAG_OWN_REALTY'
cat = 'NAME_INCOME_TYPE'
tmp = app.query('TARGET!=-1')
avg = tmp[[cat, target]].groupby(cat)[target].mean().to_frame()
size = tmp[[cat, target]].groupby(cat)[target].size().to_frame().rename(columns={target:'size'})
result = avg.join(size)
result

,TARGET,size
NAME_INCOME_TYPE,,
Businessman,0.000000,10
Commercial associate,0.074843,71617
Maternity leave,0.400000,5
Pensioner,0.053864,55362
State servant,0.057550,21703
Student,0.000000,18
Unemployed,0.363636,22
Working,0.095885,158774


In [1]:
path = '../features/5_engineering/*.npy'
data = make_feature_set(base, path)

In [19]:
df = data

'positive率とサンプル数によってラベリングする'

' SUITE'
df['a_NAME_TYPE_SUITE'] = df['a_NAME_TYPE_SUITE'].map({
    'Children' : 'middle_row_risk'
    ,'Family' : 'middle_row_risk'
    ,'Unaccompanied' : 'middle_risk'
    ,'Groupofpeople' : 'hight_risk'
    ,'Other_A' : 'high_risk'
    ,'Other_B' : 'high_risk'
    ,'Spouse,partner' : 'row_risk'
})

' INCOME_TYPE'
df['a_NAME_INCOME_TYPE'] = df['a_NAME_INCOME_TYPE'].map(lambda x:'Pensioner' if x.count('Business') or x.count('Student') else 'Working' if x.count('Unemployed') or x.count('Maternity') else x)

' EDUCATION_TYPE'
df['a_NAME_EDUCATION_TYPE'] = df['a_NAME_EDUCATION_TYPE'].map(lambda x:'Secondary / secondary special' if x.count('Lower secondary') else 'Higher education' if x.count('Academic') else x)

' EDUCATION_TYPE'
df['a_NAME_FAMILY_STATUS'] = df['a_NAME_FAMILY_STATUS'].map(lambda x:'Married' if x.count('Widow') or x.count('Unknown') else 'Civil marriage' if x.count('Single') or x.count('Separated') else x)

' ORGANIZATION '
tmp = df.query('TARGET != -1')
category = 'a_ORGANIZATION_TYPE'
# category = 'a_OCCUPATION_TYPE'
target_avg = tmp.groupby(category)['TARGET'].mean().reset_index()
print(target_avg)
sys.exit()

' positive率でカテゴリをビンにする '
bins = 10
bin_list = ['TARGET']
for col in bin_list:
    target_avg[target] = pd.qcut(x=target_avg[col], q=bins, duplicates='drop')
    target_avg.set_index(category, inplace=True)
    target_avg = target_avg.iloc[:,0]
df[category] = df[category].map(target_avg)

' positive率が低い順に番号をつける '
bin_sort = df[category].drop_duplicates().sort_values().to_frame()
bin_sort['index'] = np.arange(len(bin_sort))+1
bin_sort = bin_sort.set_index(category).iloc[:,0]
df[category] = df[category].map(bin_sort)

  a_NAME_FAMILY_STATUS    TARGET
0       Civil marriage  0.095148
1              Married  0.074284


SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [39]:
df[category].value_counts()

Married           247014
Civil marriage    109241
Name: a_NAME_FAMILY_STATUS, dtype: int64

In [152]:
category = 'NAME_EDUCATION_TYPE'
category = 'NAME_INCOME_TYPE'
category = 'NAME_TYPE_SUITE'
category = 'ORGANIZATION_TYPE'
category = 'OCCUPATION_TYPE'
df[category].value_counts()

11    96391
8     61132
2     48941
7     32102
1     21193
10    20696
5      9842
9      8069
6      5404
4      2652
3      1089
Name: a_OCCUPATION_TYPE, dtype: int64

In [46]:
method_list = ['mean', 'std']
# method_list = ['sum', 'mean', 'std', 'max', 'min']

prefix = 'a_'
val_col = 'valid_no_4'

bins = 30
bin_list = [
    'a_DAYS_BIRTH',
    'a_DAYS_EMPLOYED',
    'a_DAYS_ID_PUBLISH',
    'a_DAYS_REGISTRATION',
    'a_DAYS_LAST_PHONE_CHANGE',
    'a_EXT_SOURCE_avg@',
    'AMT_CREDIT_diff_AMT_ANNUITY@_div_AMT_ANNUITY@',
    'AMT_CREDIT_diff_AMT_ANNUITY@_div_AMT_GOODS_PRICE@',
    'a_AMT_CREDIT'
            ]

cat_list_1 = []
cat_list_1 = [
    'a_OCCUPATION_TYPE'
    ,'a_ORGANIZATION_TYPE'
    ,'a_NAME_EDUCATION_TYPE'
    ,'a_NAME_INCOME_TYPE'
    ,'a_CODE_GENDER'
    ,'a_NAME_TYPE_SUITE'
    ,'a_NAME_FAMILY_STATUS'
    ,'a_REGION_RATING_CLIENT_W_CITY'
    ,'a_HOUSE_HOLD_CODE@'
    ,'a_FLAG_WORK_PHONE'
]

cat_list_2 = [
    'a_OCCUPATION_TYPE'
    ,'a_ORGANIZATION_TYPE'
    ,'a_NAME_EDUCATION_TYPE'
    ,'a_NAME_INCOME_TYPE'
    ,'a_CODE_GENDER'
    ,'a_NAME_FAMILY_STATUS'
    ,'a_NAME_TYPE_SUITE'
    ,'a_REGION_RATING_CLIENT_W_CITY'
    ,'a_HOUSE_HOLD_CODE@'
    ,'a_FLAG_WORK_PHONE'
]

' binをカテゴリとする場合はこちらb '
# for col in bin_list:
#     df[f'bin{bins}_{col}'] = pd.qcut(x=df[col], q=bins, duplicates='drop')
#     cat_list_1.append(f'bin{bins}_{col}')
    
#     ' 可視化チェック '
#     print(df[f'bin{bins}_{col}'].value_counts())
#     df.drop(f'bin{bins}_{col}', axis=1, inplace=True)
#     sys.exit()
    

categorical = []
for cat1 in cat_list_1:
    for cat2 in cat_list_2:
        if cat1==cat2:continue
        categorical.append([cat1, cat2])

' エンコーディングするカテゴリカラムのリストを作る '
#  categorical = get_categorical_features(df, ignore_features)

' カテゴリの組み合わせをエンコーディングする場合はこちら '
#  cat_combi = list(combinations(categorical, 2))
#  categorical = cat_combi

select_list = []
#  categorical = [col for col in df.columns if col.count('cluster')]

for cat in categorical:
    target_encoding(data=df, level=cat, method_list=method_list, prefix=prefix, select_list=select_list, test=1, impute=1208, val_col=val_col)
sys.exit()

2018-07-22 23:05:59,327 logger 89 [INFO]    [target_encoding] 
df_val: (109, 2) 
2018-07-22 23:05:59,327 logger 89 [INFO]    [target_encoding] 
df_val: (109, 2) 
2018-07-22 23:05:59,327 logger 89 [INFO]    [target_encoding] 
df_val: (109, 2) 
2018-07-22 23:05:59,402 logger 92 [INFO]    [target_encoding] 
df_agg: (246009, 3) 
2018-07-22 23:05:59,402 logger 92 [INFO]    [target_encoding] 
df_agg: (246009, 3) 
2018-07-22 23:05:59,402 logger 92 [INFO]    [target_encoding] 
df_agg: (246009, 3) 
2018-07-22 23:05:59,404 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
2018-07-22 23:05:59,404 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
2018-07-22 23:05:59,404 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprec

2018-07-22 23:06:00,860 logger 118 [INFO]    [target_encoding] 
   SK_ID_CURR  SK_ID_CURR  valid_no_4  a_OCCUPATION_TYPE  a_ORGANIZATION_TYPE  \
0      100002      100002         2.0                  8                    8   
1      100003      100003         1.0                  2                    3   
2      100004      100004         5.0                  8                    4   
3      100006      100006         3.0                  8                    8   
4      100007      100007         4.0                  2                    3   

   TARGET_mean@['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']  
0                                           0.110953         
1                                           0.042069         
2                                           0.075767         
3                                           0.110373         
4                                           0.041850          
2018-07-22 23:06:00,860 logger 118 [INFO]    [target_encoding] 
   SK_ID_CUR

2018-07-22 23:06:01,151 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
2018-07-22 23:06:01,151 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
2018-07-22 23:06:01,151 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']
valid_no: 2.0 
2018-07-22 23:06:01,184 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (109, 4) 
2018-07-22 23:06:01,184 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (109, 4) 
2018-07-22 23:06:01,184 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (109, 4) 
2018-07-22 23:06:01,278 logger 89 [INFO]    [target_encoding] 
df_val: (108, 2) 
2018-07-22 23:06:01,278 logger 89 [INFO]    [target_encoding] 
df_val: (108, 2) 
2018-07-22 23:06:01,278 logger 89 [INFO]    [target_encoding] 
df_val: (108, 2) 
2018-07-22 23:06:01,350 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07

2018-07-22 23:06:02,860 logger 118 [INFO]    [target_encoding] 
   SK_ID_CURR  SK_ID_CURR  valid_no_4  a_OCCUPATION_TYPE  a_ORGANIZATION_TYPE  \
0      100002      100002         2.0                  8                    8   
1      100003      100003         1.0                  2                    3   
2      100004      100004         5.0                  8                    4   
3      100006      100006         3.0                  8                    8   
4      100007      100007         4.0                  2                    3   

   TARGET_std@['a_OCCUPATION_TYPE', 'a_ORGANIZATION_TYPE']  
0                                           0.314083        
1                                           0.200765        
2                                           0.264665        
3                                           0.313363        
4                                           0.200265         
2018-07-22 23:06:02,886 logger 29 [INFO]    [make_npy] 0.224577    11393
0.225677 

2018-07-22 23:06:03,436 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07-22 23:06:03,436 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07-22 23:06:03,436 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07-22 23:06:03,438 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_NAME_EDUCATION_TYPE']
valid_no: 1.0 
2018-07-22 23:06:03,438 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_NAME_EDUCATION_TYPE']
valid_no: 1.0 
2018-07-22 23:06:03,438 logger 94 [INFO]    [target_encoding] 
level: ['a_OCCUPATION_TYPE', 'a_NAME_EDUCATION_TYPE']
valid_no: 1.0 
2018-07-22 23:06:03,502 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (66, 4) 
2018-07-22 23:06:03,502 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (66, 4) 
2018-07-22 23:06:03,502 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (66, 4) 
2018-07-22 23:06:03,611 logger 89 [INFO]    [target_encoding] 
df_val: (33, 2) 

...

2018-07-22 23:11:05,182 logger 118 [INFO]    [target_encoding] 
   SK_ID_CURR  SK_ID_CURR  valid_no_4  a_FLAG_WORK_PHONE  a_HOUSE_HOLD_CODE@  \
0      100002      100002         2.0                  0                   1   
1      100003      100003         1.0                  0                  -2   
2      100004      100004         5.0                  1                   1   
3      100006      100006         3.0                  0                  -2   
4      100007      100007         4.0                  0                   1   

   TARGET_mean@['a_FLAG_WORK_PHONE', 'a_HOUSE_HOLD_CODE@']  
0                                           0.127347        
1                                           0.066239        
2                                           0.146393        
3                                           0.066370        
4                                           0.122487         
2018-07-22 23:11:05,182 logger 118 [INFO]    [target_encoding] 
   SK_ID_CURR  SK_ID_CUR

2018-07-22 23:11:05,643 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07-22 23:11:05,643 logger 92 [INFO]    [target_encoding] 
df_agg: (246008, 3) 
2018-07-22 23:11:05,647 logger 94 [INFO]    [target_encoding] 
level: ['a_FLAG_WORK_PHONE', 'a_HOUSE_HOLD_CODE@']
valid_no: 1.0 
2018-07-22 23:11:05,647 logger 94 [INFO]    [target_encoding] 
level: ['a_FLAG_WORK_PHONE', 'a_HOUSE_HOLD_CODE@']
valid_no: 1.0 
2018-07-22 23:11:05,647 logger 94 [INFO]    [target_encoding] 
level: ['a_FLAG_WORK_PHONE', 'a_HOUSE_HOLD_CODE@']
valid_no: 1.0 
2018-07-22 23:11:05,682 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (26, 4) 
2018-07-22 23:11:05,682 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (26, 4) 
2018-07-22 23:11:05,682 logger 105 [INFO]    [target_encoding] 
tmp_result shape: (26, 4) 
2018-07-22 23:11:05,790 logger 89 [INFO]    [target_encoding] 
df_val: (14, 2) 
2018-07-22 23:11:05,790 logger 89 [INFO]    [target_encoding] 
df_val: (14, 2) 
2018-07-22 2

2018-07-22 23:11:06,586 logger 29 [INFO]    [make_npy] -2    167015
-1     95732
-3     56120
 1     21978
 0     11990
 2      3414
-4         6
Name: a_HOUSE_HOLD_CODE@, dtype: int64 
2018-07-22 23:11:06,586 logger 29 [INFO]    [make_npy] -2    167015
-1     95732
-3     56120
 1     21978
 0     11990
 2      3414
-4         6
Name: a_HOUSE_HOLD_CODE@, dtype: int64 
2018-07-22 23:11:06,586 logger 29 [INFO]    [make_npy] -2    167015
-1     95732
-3     56120
 1     21978
 0     11990
 2      3414
-4         6
Name: a_HOUSE_HOLD_CODE@, dtype: int64 
2018-07-22 23:11:06,603 logger 29 [INFO]    [make_npy] 0.248324    22946
0.248929    22878
0.248701    22698
0.247505    22649
0.248086    22546
0.248309    18919
0.287317    13099
0.286746    13058
0.287455    12989
0.285896    12905
0.286388    12759
0.286760     9923
0.243486     8453
0.241957     8445
0.241860     8400
0.243480     8388
0.244497     8343
0.243059     6135
0.282434     5940
0.281596     5923
0.281858     5914
0.281036 

SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [112]:
' 生涯賃金を計算 '
app['DAYS_WORK'] = (app['DAYS_BIRTH']) +240
app['DAYS_WORK'] = abs(app['DAYS_WORK'].clip(-540, 0))
app['INCOME_LIFE_TOTAL@'] = app['DAYS_WORK'] * app['AMT_INCOME_TOTAL']

In [119]:
# days_list = [col for col in app.columns if col.count('DAYS')]
# for days in days_list:
#     log_num = app[days].values
#     log_num = log_num - log_num.min() +1
#     log_num = np.log10(log_num)
#     app[f'{days}_log10'] = log_num

amt_list = [
#     'AMT_CREDIT' , 'AMT_ANNUITY'
            'AMT_INCOME_TOTAL'
           ]
for amt in amt_list:
#     app = division_feature(df=app, first=amt, second='INCOME_LIFE_TOTAL@')
#     app = division_feature(df=app, first=amt, second='AMT_INCOME_TOTAL')
    app = division_feature(df=app, first=amt, second='DAYS_BIRTH_log10')
    app = division_feature(df=app, first=amt, second='DAYS_ID_PUBLISH_log10')
    app = division_feature(df=app, first=amt, second='DAYS_EMPLOYED_log10')
    app = division_feature(df=app, first=amt, second='DAYS_REGISTRATION_log10')
    app = division_feature(df=app, first=amt, second='DAYS_LAST_PHONE_CHANGE_log10')

In [116]:
days_list.remove('DAYS_WORK')
for days_1 in days_list:
    for days_2 in days_list:
        if days_1==days_2:continue
        app = division_feature(df=app, first=days_1, second=days_2)

In [18]:
app_cat_list = get_categorical_features(app, ignore_features)
for i in app_cat_list:
    if i.count('FLAG') or (i in cat_list):
        app_cat_list.remove(i)
app_cat_list.remove('ORGANIZATION_TYPE')
app_cat_list.remove('FLAG_OWN_REALTY')
app_cat_list.remove('CODE_GENDER')
app_cat_list

['NAME_CONTRACT_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'NAME_TYPE_SUITE',
 'WEEKDAY_APPR_PROCESS_START']

In [42]:
df_2 = app.copy()
# docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
# live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
area_avg = [_f for _f in df_2.columns if ('_AVG' in _f) and not(_f.count('YEARS_'))]
area_medi = [_f for _f in df_2.columns if ('_MEDI' in _f) and not(_f.count('YEARS_'))]
area_mode = [_f for _f in df_2.columns if ('_MODE' in _f) and not(_f.count('YEARS_'))]

df_2['DIFF_AMT_REQ_CREDIT_BUREAU@'] = df_2['AMT_REQ_CREDIT_BUREAU_YEAR'] - df_2['AMT_REQ_CREDIT_BUREAU_QRT']
df_2['DIFF_DEF_CNT_SOCIAL_CIRCLE@'] = df_2['DEF_30_CNT_SOCIAL_CIRCLE'] - df_2['DEF_60_CNT_SOCIAL_CIRCLE']
df_2['DIFF_OBS_CNT_SOCIAL_CIRCLE@'] = df_2['OBS_30_CNT_SOCIAL_CIRCLE'] - df_2['OBS_60_CNT_SOCIAL_CIRCLE']
sys.exit()

# inc_by_org = df_2[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

df_2['AREA_AVG_sum@'] = df_2[area_avg].fillna(0).sum(axis=1)
df_2['AREA_MEDI_sum@'] = df_2[area_medi].fillna(0).sum(axis=1)
df_2['AREA_MODE_sum@'] = df_2[area_mode].fillna(0).sum(axis=1)

# df_2['FLG_DOCUMENT_KURT@'] = df_2[docs].kurtosis(axis=1)
# df_2['FLG_VARIETY@'] = df_2[live].sum(axis=1)
# df_2['INCOME_AT_ORGANIZATION@'] = df_2['ORGANIZATION_TYPE'].map(inc_by_org)
# df_2['INCOME_per_CHILD@'] = df_2['AMT_INCOME_TOTAL'] / (df_2['CNT_CHILDREN']+1)

SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [43]:
for col in df_2.columns:
    if col.count('@') and col.count('DIFF_'):
        np.save(f'../features/1_first_valid/{col}', df_2[col].values)

In [28]:
cnt_list =  ['CNT_FAM_MEMBERS', 'CNT_CHILDREN']
app['CNT_CHILDREN'] +=1

for amt in amt_list:
    for cnt in cnt_list:
        app = division_feature(app, amt, cnt)
        
for day in days_list:
    for cnt in cnt_list:
        app = division_feature(app, amt, cnt)

In [11]:
' 世帯属性 '
app = household_code(df=app)
# app

' 世帯属性 '